In [1]:
import cv2 as cv
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_eye.xml')
left_eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
right_eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_righteye_2splits.xml')

In [2]:
def getextractedeyes(image_path, save_path="D:/projects/Driver_Drowziness/results/"):
    img = cv.imread(image_path)
    name = image_path[-13:-8]
    print(name)
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)

    if len(faces) == 0:
        print("No faces detected!")
        return

    for (x, y, w, h) in faces:
        roi_gray = gray_img[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

        left_half = roi_gray[:, :w//2]
        right_half = roi_gray[:, w//2:]

        ans = []
        
        roi_height = min(left_half.shape[0], right_half.shape[0])
        roi_width = min(left_half.shape[1], right_half.shape[1])

        left_roi = left_half[:roi_height, :roi_width]
        right_roi = right_half[:roi_height, :roi_width]

        left_eyes = eye_cascade.detectMultiScale(left_roi)
        for i, (ex, ey, ew, eh) in enumerate(left_eyes):
            left_eye = roi_color[ey:ey+eh, ex:ex+ew]
            left_eye = cv.resize(left_eye, (145, 145))
            ans.append(left_eye)
            respathl = f"{save_path}/{name}_left.jpg"
            cv.imwrite(respathl, left_eye)

        right_eyes = eye_cascade.detectMultiScale(right_roi)
        for i, (ex, ey, ew, eh) in enumerate(right_eyes):
            ex += w//2
            right_eye = roi_color[ey:ey+eh, ex:ex+ew]
            right_eye = cv.resize(right_eye, (145, 145))
            ans.append(right_eye)
            respathr = f"{save_path}/{name}_right.jpg"
            cv.imwrite(respathr, right_eye)

    return ans

In [3]:
Path = r"C:\Users\vkris\Pictures\Camera Roll\WIN_20231122_14_31_04_Pro.jpg"
eyes = getextractedeyes(Path)

31_04


In [4]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [6]:
model = load_model(r"trained\drowsiness_detection_model_final.h5")

In [7]:
def preprocess_eye_image(img):
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

In [8]:
lefteye_detected = False
righteye_detected = False

for i, eye in enumerate(eyes):
    preprocessed_eye = preprocess_eye_image(eye)
    pred = model.predict(preprocessed_eye)
    print(pred)
    if pred[0, 0] > 0.5:
        if i == 0:
            lefteye_detected = False
        else:
            righteye_detected = False
    else:
        if i == 0:
            lefteye_detected = True
        else:
            righteye_detected = True

if lefteye_detected and righteye_detected:
    print("True, drowsiness detected!")
else:
    print("No Drowsiness Detected")


1/1 [==============================] - 0s 285ms/step
[[0.01134036]]
1/1 [==============================] - 0s 45ms/step
[[0.01354509]]
True, drowsiness detected!
